In [ ]:
# !pip -q install -U python-dotenv
# !pip -q install pydantic-ai
# !pip -q install nest_asyncio
# !pip -q install devtools
# !pip -q install tavily-python

## API Keys setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv("keys.env")

True

In [2]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_SEARCH_API_KEY"]

## Tavily Search

In [4]:
from tavily import TavilyClient, AsyncTavilyClient
tavily_client = AsyncTavilyClient(api_key=TAVILY_API_KEY)

# Sample Search
response = await tavily_client.search("What is PydanticAI?", max_results=3)
print(response['results'])

[{'title': 'PydanticAI Introduction: What is PydanticAI about', 'url': 'https://aipure.ai/products/pydantic-ai/introduction', 'content': "PydanticAI is a Python Agent Framework that streamlines the development of production-grade AI applications by combining Pydantic's powerful data validation with LLM integration, offering type-safe dependency injection and model-agnostic support. PydanticAI is an innovative agent framework developed by the team behind Pydantic, designed to simplify the process of building production-grade applications with Generative AI. The framework also supports streamed responses and can validate structured responses using Pydantic models, making it particularly powerful for complex AI applications that require reliable data handling. CopilotForXcode is an Xcode Source Editor Extension that integrates GitHub Copilot, Codeium, and ChatGPT to provide AI-powered code suggestions, chat assistance, and prompt-to-code functionality within Xcode. BLACKBOX AI is an AI-po

In [5]:
# Responses
for result in response['results']:
    print(result['title'])
    print(result['content'])
    print('\n')

PydanticAI Introduction: What is PydanticAI about
PydanticAI is a Python Agent Framework that streamlines the development of production-grade AI applications by combining Pydantic's powerful data validation with LLM integration, offering type-safe dependency injection and model-agnostic support. PydanticAI is an innovative agent framework developed by the team behind Pydantic, designed to simplify the process of building production-grade applications with Generative AI. The framework also supports streamed responses and can validate structured responses using Pydantic models, making it particularly powerful for complex AI applications that require reliable data handling. CopilotForXcode is an Xcode Source Editor Extension that integrates GitHub Copilot, Codeium, and ChatGPT to provide AI-powered code suggestions, chat assistance, and prompt-to-code functionality within Xcode. BLACKBOX AI is an AI-powered coding assistant that helps developers write, debug, and optimize code faster thro

## Agents and Tools

In [6]:
from __future__ import annotations as _annotations

import asyncio
from dataclasses import dataclass
from typing import Any

from devtools import debug
from httpx import AsyncClient
import datetime

from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic import BaseModel, Field
import pandas as pd
from typing import Any, List, Dict

In [7]:
@dataclass
class SearchDataclass:
    max_results: int
    todays_date: str

@dataclass
class ResearchDependencies:
    todays_date: str

class ResearchResult(BaseModel):
    research_title: str = Field(description='This is a top-level Markdown heading that covers the topic of the query and answer, prefixed with #')
    research_main: str = Field(description='This is a main section that provides answers for the query and research')
    research_numbers: str = Field(description='This is a section of numeric data related to the research')
    research_bullets: str = Field(description='This is a set of bullet points that summarize the answers for the query')

In [8]:
## Create the agent
research_agent = Agent('openai:gpt-4o',
                     deps_type=ResearchDependencies,
                     result_type=ResearchResult,
                     system_prompt='Your a helpful research assistant, you are an expert in research '
                     'If you are given a question you write strong keywords to do 3-5 searches in total '
                     '(each with a query_number) and then combine the results', )

In [9]:
@research_agent.tool #Tavily
async def get_search(search_data:RunContext[SearchDataclass],query: str, query_number: int) -> dict[str, Any]:
    """Get the search for a keyword query.

    Args:
        query: keywords to search.
    """
    print(f"Search query {query_number}: {query}")
    max_results = search_data.deps.max_results
    results = await tavily_client.get_search_context(query=query, max_results=max_results)

    return results

In [10]:
# Get the current date
current_date = datetime.date.today()
date_string = current_date.strftime("%Y-%m-%d")
deps = SearchDataclass(max_results=3, todays_date=date_string)

In [11]:
result = await research_agent.run(
    'How is the adoption of Gen AI growing in Tech Companies in India?', deps=deps
)

Search query 1: Gen AI adoption in tech companies India 2023
Search query 2: Growth of generative AI in Indian technology sector
Search query 3: Technology companies in India using generative AI


In [12]:
print(result.data)

research_title='Adoption of Generative AI in Indian Tech Companies' research_main="The adoption of Generative AI (Gen AI) in tech companies across India has seen significant growth in recent years. As of 2023, India is home to over 60 startups focused primarily on Gen AI, reflecting a dynamic and rapidly evolving sector. The market's momentum is propelled by easier-to-use tools aimed at cost reduction and process automation, as identified in the 2023 IBM Global AI Adoption Index. Active deployment of AI technologies stands at approximately 59% among large enterprises in India, marking the country's leading position in adoption rates. This expansion is not only limited to startups but also includes large enterprises that are increasingly embedding Gen AI into their operational infrastructure.\n\nThe Generative AI market in India is forecasted to grow substantially, with an estimated growth rate of 46.32% from 2024 to 2030, reaching a market volume of $6.28 billion. This growth is expect

In [13]:
result.data.research_title = '#'+result.data.research_title
print(result.data.research_title)

#Adoption of Generative AI in Indian Tech Companies


In [14]:
print(result.data.research_main)

The adoption of Generative AI (Gen AI) in tech companies across India has seen significant growth in recent years. As of 2023, India is home to over 60 startups focused primarily on Gen AI, reflecting a dynamic and rapidly evolving sector. The market's momentum is propelled by easier-to-use tools aimed at cost reduction and process automation, as identified in the 2023 IBM Global AI Adoption Index. Active deployment of AI technologies stands at approximately 59% among large enterprises in India, marking the country's leading position in adoption rates. This expansion is not only limited to startups but also includes large enterprises that are increasingly embedding Gen AI into their operational infrastructure.

The Generative AI market in India is forecasted to grow substantially, with an estimated growth rate of 46.32% from 2024 to 2030, reaching a market volume of $6.28 billion. This growth is expected to transform various sectors, with significant financial impacts forecasted, parti

In [15]:
print(result.data.research_numbers)

- Over 60 Gen AI-focused startups in India as of 2023.
- 59% of large Indian enterprises have actively deployed AI.
- Generative AI market projected to grow by 46.32% (2024-2030).
- Expected market size of Gen AI in India to reach $6.28 billion by 2030.


In [16]:
print(result.data.research_bullets)

- Significant growth in Gen AI startups in India, totaling over 60.
- High rate of AI deployment among large enterprises at 59%.
- Forecasted 46.32% growth rate in India's Gen AI market through 2030.
- Potential market volume of $6.28 billion by 2030.
- Focus on digital transformation and R&D investment in AI by Indian companies.


In [17]:
from IPython.display import Markdown

In [18]:
combined_markdown = "\n\n".join([result.data.research_title, result.data.research_main, result.data.research_numbers, result.data.research_bullets])
Markdown(combined_markdown)

#Adoption of Generative AI in Indian Tech Companies

The adoption of Generative AI (Gen AI) in tech companies across India has seen significant growth in recent years. As of 2023, India is home to over 60 startups focused primarily on Gen AI, reflecting a dynamic and rapidly evolving sector. The market's momentum is propelled by easier-to-use tools aimed at cost reduction and process automation, as identified in the 2023 IBM Global AI Adoption Index. Active deployment of AI technologies stands at approximately 59% among large enterprises in India, marking the country's leading position in adoption rates. This expansion is not only limited to startups but also includes large enterprises that are increasingly embedding Gen AI into their operational infrastructure.

The Generative AI market in India is forecasted to grow substantially, with an estimated growth rate of 46.32% from 2024 to 2030, reaching a market volume of $6.28 billion. This growth is expected to transform various sectors, with significant financial impacts forecasted, particularly within technology services. Indian enterprises view Gen AI as a critical component in accelerating digital transformation, revealing substantial economic opportunities. However, challenges such as data privacy concerns and the high computational costs associated with Gen AI deployment persist.

Furthermore, organizations recognized as AI high performers have been quick to adopt Gen AI tools, prioritizing new business generation and enhancement of existing services through AI integration. This rapid adoption is reflected in their ongoing investment in research and development, as well as workforce upskilling initiatives, responding to the industry's pressing demand for skilled personnel.

- Over 60 Gen AI-focused startups in India as of 2023.
- 59% of large Indian enterprises have actively deployed AI.
- Generative AI market projected to grow by 46.32% (2024-2030).
- Expected market size of Gen AI in India to reach $6.28 billion by 2030.

- Significant growth in Gen AI startups in India, totaling over 60.
- High rate of AI deployment among large enterprises at 59%.
- Forecasted 46.32% growth rate in India's Gen AI market through 2030.
- Potential market volume of $6.28 billion by 2030.
- Focus on digital transformation and R&D investment in AI by Indian companies.

## Research Without using Date

In [23]:
result = await research_agent.run(
    'What is the latest AI News?', deps=deps
)

Search query 1: latest AI news December 2024
Search query 2: AI technology developments December 2024
Search query 3: artificial intelligence research updates December 2024
Search query 4: recent advancements in AI December 2024


In [24]:
print(result.data)

research_title='Latest Developments in AI: December 2024' research_main="December 2024 has seen numerous advancements and discussions in the field of artificial intelligence, ranging from technological breakthroughs to regulatory challenges. A significant highlight includes the continued evolution of large language models (LLMs) like Google’s Gemini and Claude by Anthropic, although achieving artificial general intelligence (AGI) remains elusive. Multimodal AI, leveraging various modes of input beyond text, is gaining traction, highlighting potential future interactions.\n\nAI's integration across various industries continues to accelerate. Specialized AI applications are being developed tailored to specific sectors, reflecting a shift from general-purpose to more targeted solutions. The year has witnessed rapid deployment of AI technologies, transforming sectors like healthcare, where AI breakthroughs are catalyzing innovation.\n\nRegulatory frameworks are evolving as governments reac

## Research With Date in System Prompt

In [26]:
@research_agent.system_prompt
async def add_current_date(ctx: RunContext[ResearchDependencies]) -> str:
    todays_date = ctx.deps.todays_date
    system_prompt=f'Your a helpful research assistant, you are an expert in research \
                If you are given a question you write strong keywords to do 3-5 searches in total \
                (each with a query_number) and then combine the results \
                if you need todays date it is {todays_date}'
    return system_prompt

In [27]:
result = await research_agent.run(
    'What are the major AI libraries released in the last few weeks?', deps=deps
)

Search query 1: major AI libraries releases November December 2024
Search query 2: new AI libraries November December 2024
Search query 3: latest AI libraries 2024


In [28]:
print(result.data)

research_title='Recent Releases of Major AI Libraries (November-December 2024)' research_main="Recent months have seen significant advancements and releases of AI libraries that are contributing to the ongoing evolution of artificial intelligence technologies. Notably, December 2024 marked the accelerated integration of AWS Trainium accelerators within JAX, enhancing its utility and flexibility in AI infrastructure by democratizing machine learning capabilities. This integration provides developers with more portability and choices in deploying machine learning models.\n\nFurther enhancement in AI functionalities can be seen with OpenAI's imminent release of the Orion AI model, which is expected to become available in December 2024. Orion promises significant improvements, potentially offering up to 100 times the power of GPT-4. This continuation of expansion demonstrates a profound push toward more powerful AI models available for developers.\n\nIn 2024, legacy libraries such as Tenso

In [29]:
result.data.research_title = '#'+result.data.research_title
print(result.data.research_title)

#Recent Releases of Major AI Libraries (November-December 2024)


In [30]:
print(result.data.research_main)

Recent months have seen significant advancements and releases of AI libraries that are contributing to the ongoing evolution of artificial intelligence technologies. Notably, December 2024 marked the accelerated integration of AWS Trainium accelerators within JAX, enhancing its utility and flexibility in AI infrastructure by democratizing machine learning capabilities. This integration provides developers with more portability and choices in deploying machine learning models.

Further enhancement in AI functionalities can be seen with OpenAI's imminent release of the Orion AI model, which is expected to become available in December 2024. Orion promises significant improvements, potentially offering up to 100 times the power of GPT-4. This continuation of expansion demonstrates a profound push toward more powerful AI models available for developers.

In 2024, legacy libraries such as TensorFlow and PyTorch continue to dominate the machine learning landscape, providing robust frameworks 

In [31]:
print(result.data.research_numbers)

- TensorFlow and PyTorch are documented as the dominant AI libraries in 2024.
- OpenAI's Orion is posited to be up to 100 times more powerful than GPT-4.
- AI libraries have seen continued innovation with approximately $13.8 billion in AI-focused investments projected by Gen AI reports.


In [32]:
print(result.data.research_bullets)

- A new AI library integration involves AWS Trainium accelerators into JAX, enhancing AI model portability and choice.
- OpenAI's Orion model is set for release in December 2024, expected to exceed earlier models like GPT-4 in power.
- TensorFlow and PyTorch remain dominant AI libraries, critical for developing AI models in 2024.
- The AI ecosystem in 2024 shifts rapidly, with substantial emphasis on staying updated with industry trends to leverage emerging tools effectively.


In [33]:
combined_markdown = "\n\n".join([result.data.research_title, result.data.research_main, result.data.research_numbers, result.data.research_bullets])
Markdown(combined_markdown)

#Recent Releases of Major AI Libraries (November-December 2024)

Recent months have seen significant advancements and releases of AI libraries that are contributing to the ongoing evolution of artificial intelligence technologies. Notably, December 2024 marked the accelerated integration of AWS Trainium accelerators within JAX, enhancing its utility and flexibility in AI infrastructure by democratizing machine learning capabilities. This integration provides developers with more portability and choices in deploying machine learning models.

Further enhancement in AI functionalities can be seen with OpenAI's imminent release of the Orion AI model, which is expected to become available in December 2024. Orion promises significant improvements, potentially offering up to 100 times the power of GPT-4. This continuation of expansion demonstrates a profound push toward more powerful AI models available for developers.

In 2024, legacy libraries such as TensorFlow and PyTorch continue to dominate the machine learning landscape, providing robust frameworks that support a wide range of AI applications. Each offers unique capabilities and remains critical for the development of sophisticated AI models.

Amidst the technological advancements, industry experts recommend developers to stay engaged with AI news and blogs to harness the full potential of these evolving tools.

- TensorFlow and PyTorch are documented as the dominant AI libraries in 2024.
- OpenAI's Orion is posited to be up to 100 times more powerful than GPT-4.
- AI libraries have seen continued innovation with approximately $13.8 billion in AI-focused investments projected by Gen AI reports.

- A new AI library integration involves AWS Trainium accelerators into JAX, enhancing AI model portability and choice.
- OpenAI's Orion model is set for release in December 2024, expected to exceed earlier models like GPT-4 in power.
- TensorFlow and PyTorch remain dominant AI libraries, critical for developing AI models in 2024.
- The AI ecosystem in 2024 shifts rapidly, with substantial emphasis on staying updated with industry trends to leverage emerging tools effectively.